In [9]:
%store -r df_2


In [10]:
df_2.shape

(988, 21)

In [11]:
df_2.head()

,Cle,Comptes,Duree_credit,Historique_credit,Objet_credit,Montant_credit,Epargne,Anciennete_emploi,Taux_effort,Situation_familiale,...,Anciennete_domicile,Biens,Age,Autres_credits,Statut_domicile,Nb_credits,Type_emploi,Nb_pers_charge,Telephone,Cible
0,1.0,1.0,6.0,A34,A43,1169.0,0.0,5.0,4.0,A93,...,4.0,A121,67.0,A143,A152,2.0,A173,1.0,A192,0
1,2.0,2.0,48.0,A32,A43,5951.0,1.0,3.0,2.0,A92,...,2.0,A121,22.0,A143,A152,1.0,A173,1.0,A191,1
2,3.0,4.0,12.0,A34,A46,2096.0,1.0,4.0,2.0,A93,...,3.0,A121,49.0,A143,A152,1.0,A172,2.0,A191,0
3,4.0,1.0,42.0,A32,A42,7882.0,1.0,4.0,2.0,A93,...,4.0,A122,45.0,A143,A153,1.0,A173,2.0,A191,0
4,5.0,1.0,24.0,A33,A40,4870.0,1.0,3.0,3.0,A93,...,4.0,A124,53.0,A143,A153,2.0,A173,2.0,A191,1


In [12]:
df_2.info()


<class 'pandas.core.frame.DataFrame'>
Index: 988 entries, 0 to 999
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   Cle                  988 non-null    category
 1   Comptes              988 non-null    category
 2   Duree_credit         988 non-null    float64 
 3   Historique_credit    988 non-null    category
 4   Objet_credit         988 non-null    category
 5   Montant_credit       988 non-null    float64 
 6   Epargne              988 non-null    category
 7   Anciennete_emploi    988 non-null    category
 8   Taux_effort          988 non-null    category
 9   Situation_familiale  988 non-null    category
 10  Garanties            988 non-null    category
 11  Anciennete_domicile  988 non-null    category
 12  Biens                988 non-null    category
 13  Age                  988 non-null    float64 
 14  Autres_credits       988 non-null    category
 15  Statut_domicile      988 non

In [14]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

# Fonction pour calculer Cramér's V
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
    rcorr = r - ((r - 1) ** 2) / (n - 1)
    kcorr = k - ((k - 1) ** 2) / (n - 1)
    return np.sqrt(phi2corr / min((kcorr - 1), (rcorr - 1)))

# Calcul uniquement pour les colonnes de type 'category'
cramers_v_values = {}
for col in df_2.select_dtypes(include=['category']).columns:
    if col != 'Cible':
        cramers_v_values[col] = cramers_v(df_2[col], df_2['Cible'])

# Tri décroissant
cramers_v_values_sorted = dict(sorted(cramers_v_values.items(), key=lambda item: item[1], reverse=True))

cramers_v_values_sorted


C:\Users\Remih\AppData\Local\Temp\ipykernel_20168\1223089768.py:15: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(phi2corr / min((kcorr - 1), (rcorr - 1)))


{'Cle': np.float64(inf),
 'Comptes': np.float64(0.3475963377775544),
 'Historique_credit': np.float64(0.23282570670092298),
 'Epargne': np.float64(0.17687570258798707),
 'Objet_credit': np.float64(0.1489412915567852),
 'Biens': np.float64(0.1404448680659372),
 'Statut_domicile': np.float64(0.12578225738983098),
 'Anciennete_emploi': np.float64(0.11423630499512544),
 'Autres_credits': np.float64(0.10203911977661499),
 'Situation_familiale': np.float64(0.08444305008215512),
 'Garanties': np.float64(0.07260483135453556),
 'Taux_effort': np.float64(0.06102017275921761),
 'Telephone': np.float64(0.020150332932422257),
 'Nb_credits': np.float64(0.01515235107575194),
 'Anciennete_domicile': np.float64(0.0),
 'Type_emploi': np.float64(0.0),
 'Nb_pers_charge': np.float64(0.0)}

In [18]:
# Supprimer les variables sans association significative (<0.1)
import math

# Supprimer la variable 'Cle' et les valeurs infinies
cramers_v_filtered = {k: v for k, v in cramers_v_values.items() if k != 'Cle' and not math.isinf(v)}

# Trier par ordre décroissant et ne garder que les 12 premières
cramers_v_values_sorted = dict(
    sorted(cramers_v_filtered.items(), key=lambda item: item[1], reverse=True)[:12]
)

cramers_v_values_sorted



{'Comptes': np.float64(0.3475963377775544),
 'Historique_credit': np.float64(0.23282570670092298),
 'Epargne': np.float64(0.17687570258798707),
 'Objet_credit': np.float64(0.1489412915567852),
 'Biens': np.float64(0.1404448680659372),
 'Statut_domicile': np.float64(0.12578225738983098),
 'Anciennete_emploi': np.float64(0.11423630499512544),
 'Autres_credits': np.float64(0.10203911977661499),
 'Situation_familiale': np.float64(0.08444305008215512),
 'Garanties': np.float64(0.07260483135453556),
 'Taux_effort': np.float64(0.06102017275921761),
 'Telephone': np.float64(0.020150332932422257)}

In [24]:
import pandas as pd
from scipy.stats import chi2_contingency

# Exemple de données
# Chaque colonne représente une variable catégorielle


# Créer un tableau de contingence
contingency_table = pd.crosstab(df_2['Comptes'], df_2['Duree_credit'])
print("Tableau de contingence :\n", contingency_table)

# Calcul du test de Khi-deux
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"\nChi2 = {chi2}")
print(f"p-value = {p}")


# Interprétation
if p < 0.05:
    print("\nIl y a une dépendance significative entre les variables.")
else:
    print("\nAucune dépendance significative entre les variables.")





Tableau de contingence :
 Duree_credit  4.0   5.0   6.0   7.0   8.0   9.0   10.0  11.0  12.0  13.0  ...  \
Comptes                                                                   ...   
1.0              0     0    22     0     3     8     4     2    52     1  ...   
2.0              0     0    14     3     4    20     3     3    43     2  ...   
3.0              1     0     7     0     0     3     6     0    13     0  ...   
4.0              5     1    32     2     0    18    15     4    68     1  ...   

Duree_credit  33.0  36.0  39.0  40.0  42.0  45.0  47.0  48.0  54.0  60.0  
Comptes                                                                   
1.0              1    21     1     1     5     2     1    14     0     2  
2.0              0    23     2     0     2     3     0    19     0     6  
3.0              0     4     0     0     2     0     0     0     0     0  
4.0              2    33     2     0     2     0     0    13     1     4  

[4 rows x 32 columns]

Chi2 = 108.63

In [26]:
import pandas as pd
from scipy.stats import chi2_contingency
import numpy as np

# Exemple : charger ton dataset
# df_2 = pd.read_csv("ton_fichier.csv")

# Colonne à exclure (clé)
key_col = 'Cle'

# Identifier les colonnes catégorielles en excluant la clé
categorical_cols = df_2.select_dtypes(include=['object', 'category']).columns
categorical_cols = [col for col in categorical_cols if col != key_col]

# Créer une matrice vide pour stocker les p-values
pval_matrix = pd.DataFrame(np.ones((len(categorical_cols), len(categorical_cols))),
                           index=categorical_cols, columns=categorical_cols)

# Boucler sur toutes les paires de variables catégorielles
for i in range(len(categorical_cols)):
    for j in range(i, len(categorical_cols)):
        col1 = categorical_cols[i]
        col2 = categorical_cols[j]
        
        # Créer le tableau de contingence
        contingency_table = pd.crosstab(df_2[col1], df_2[col2])
        
        # Test de Khi-deux
        chi2, p, dof, expected = chi2_contingency(contingency_table)
        
        # Remplir la matrice symétrique
        pval_matrix.loc[col1, col2] = p
        pval_matrix.loc[col2, col1] = p

# Affichage de la matrice des p-values
print("\nMatrice des p-values pour toutes les variables catégorielles :")
print(pval_matrix)

# Optionnel : mettre en évidence les dépendances significatives
signif_matrix = pval_matrix < 0.05
print("\nMatrice des dépendances significatives (True = p < 0.05) :")
print(signif_matrix)



Matrice des p-values pour toutes les variables catégorielles :
                          Comptes  Historique_credit  Objet_credit  \
Comptes              0.000000e+00       6.060185e-08  2.191647e-04   
Historique_credit    6.060185e-08       0.000000e+00  4.875788e-10   
Objet_credit         2.191647e-04       4.875788e-10  0.000000e+00   
Epargne              1.935384e-13       2.791185e-01  1.535428e-02   
Anciennete_emploi    9.916219e-03       1.418029e-03  4.812594e-02   
Taux_effort          4.960911e-01       7.150740e-01  2.309896e-04   
Situation_familiale  1.885229e-01       1.788092e-02  5.489815e-06   
Garanties            7.888017e-04       6.301000e-02  6.614651e-02   
Anciennete_domicile  5.803422e-04       9.651513e-02  1.141704e-02   
Biens                3.555559e-02       1.046253e-01  7.150121e-16   
Autres_credits       7.369530e-01       1.020273e-16  2.227651e-01   
Statut_domicile      2.868750e-03       2.250824e-02  4.251150e-11   
Nb_credits           1.342

In [30]:
import pandas as pd
import numpy as np

# Fonction pour calculer WoE et IV d'une colonne
def woe_iv(df, feature, target):
    lst = []
    for val in df[feature].unique():
        total = df[df[feature] == val].shape[0]
        good = df[(df[feature] == val) & (df[target] == 0)].shape[0]
        bad = df[(df[feature] == val) & (df[target] == 1)].shape[0]
        dist_good = good / df[df[target] == 0].shape[0] if df[df[target] == 0].shape[0] != 0 else 0
        dist_bad = bad / df[df[target] == 1].shape[0] if df[df[target] == 1].shape[0] != 0 else 0
        woe = np.log((dist_good + 0.0001) / (dist_bad + 0.0001))  # éviter div par zéro
        iv = (dist_good - dist_bad) * woe
        lst.append({'Value': val, 'WoE': woe, 'IV': iv})
    df_woe = pd.DataFrame(lst)
    iv_total = df_woe['IV'].sum()
    return iv_total

# Calcul de l'IV pour toutes les colonnes catégorielles sauf la cible
iv_values = {}
for col in df_2.select_dtypes(include=['category']).columns:
    if col not in ['Cible','Cle']:
        iv_values[col] = woe_iv(df_2, col, 'Cible')

# Tri décroissant
iv_values_sorted = dict(sorted(iv_values.items(), key=lambda item: item[1], reverse=True))

iv_values_sorted


{'Comptes': np.float64(0.6622986900545594),
 'Historique_credit': np.float64(0.2763503848056874),
 'Epargne': np.float64(0.19126862842095102),
 'Objet_credit': np.float64(0.151095647020873),
 'Biens': np.float64(0.10779268324807179),
 'Statut_domicile': np.float64(0.08189491302549125),
 'Anciennete_emploi': np.float64(0.08034863915716083),
 'Autres_credits': np.float64(0.05594378008191621),
 'Situation_familiale': np.float64(0.04728188495682296),
 'Garanties': np.float64(0.034636715623696364),
 'Taux_effort': np.float64(0.03263175057539598),
 'Nb_credits': np.float64(0.011270573940509789),
 'Type_emploi': np.float64(0.007721543621324756),
 'Telephone': np.float64(0.007687831596297529),
 'Anciennete_domicile': np.float64(0.002189141558812773),
 'Nb_pers_charge': np.float64(3.973742855010734e-05)}